In [1]:
using HCubature

include("src/Spectrum.jl")
include("src/ModeSolutions.jl")

view_field (generic function with 1 method)

In [12]:
function integral_region(lambda,l_num, m_num, n, R, mode, field_type; error=1e-2)
    r_border = first_zeros_bsolver(r->norm(field(r, pi/2, lambda, l_num, m_num, n, R, mode, field_type))-error, [0, R], accuracy=5)
    if m_num>2
        theta_border = first_zeros_bsolver(theta->norm(field(50, theta, lambda, l_num, m_num, n, R, mode, field_type))-error, [0, pi/2], accuracy=0.5)
    else
        theta_border = 0
    end
    if r_border == nothing && theta_border != nothing
        r_border = first_zeros_bsolver(r->norm(field(r, pi/2, lambda, l_num, m_num, n, R, mode, "E"))-error, [0, R], accuracy=5)
    end
    return r_border, theta_border
end

function mutual_region(b1, b2)
    return maximum([b1[1], b2[1]]), maximum([b1[2], b2[2]])
end

function overlap_field(field_parameters, n, R; R_region=R, digit=3)
    error = 10.0^(-floor(Int, digit/2)-1)
    rtol = 10.0^(-digit)
    lambda1, l_num1, m_num1, mode1, field_type1 = field_parameters[1]
    lambda2, l_num2, m_num2, mode2, field_type2 = field_parameters[2]
    region1 = integral_region(lambda1,l_num1, m_num1, n, R, mode1, field_type1, error=error)
    region2 = integral_region(lambda2,l_num2, m_num2, n, R, mode2, field_type2, error=error)
    if region1[1] != nothing && region1[2] != nothing && region2[1] != nothing && region2[2] != nothing
        mregion = mutual_region(region1, region2)
        region = [mregion[1], mregion[2], R_region, pi/2]
        print(region)
        f1(p) = conj(field(p[1], p[2], lambda1, l_num1, m_num1, n, R, mode1, field_type1))
        f2(p) = field(p[1], p[2], lambda2, l_num2, m_num2, n, R, mode2, field_type2)
        f3(p) = p[1]^2*sin(p[2])*f1(p)*f2(p)
        integration = hcubature(f3, region[1:2], region[3:4], rtol=rtol)
        return round(4*pi*integration[1], sigdigits=digit)
    else
        return 0
    end
end


overlap_field (generic function with 1 method)

In [16]:
lambda1 = 1502.0244243986813
l_num1 = 302
m_num1 = 302
# first_zeros_bsolver(f, range, accuracy)
R = 50
n = 1.5
mode1 = "TE"
field_type1 = "E_phi"

lambda2 = 1307.784438562213
l_num2 = 348
m_num2 = 302
# first_zeros_bsolver(f, range, accuracy)
mode2 = "TE"
field_type2 = "E_phi"

"E_phi"

In [15]:
field_parameters = [[lambda1, l_num1, m_num1, mode1, field_type1], [lambda2, l_num2, m_num2, mode2, field_type2]]
overlap_field(field_parameters, n, R)

[44.63393025240759, 1.3361136438973862, 50.0, 1.5707963267948966]

-338.0 + 0.0im